In [1]:
import pandas as pd
import json
from fractions import Fraction

In [2]:
# fixed parameters
balls = 10
name_cols = ['B','W','P','G']
name_urns = ['A','B','C','D']

# Experimental Design: Ball compositions and define the maximum number of urns = 4 and maximum number of colours = 4
ball_comp = {(2,2):{'p_col1_u1':0.7, 'p_col2_u1':0.3, 'p_col1_u2':0.3, 'p_col2_u2':0.7}, 
             (2,3):{'p_col1_u1':0.7, 'p_col2_u1':0.2, 'p_col3_u1':0.1, 'p_col1_u2':0.2, 'p_col2_u2':0.1, 'p_col3_u2':0.7},
             (2,4):{'p_col1_u1':0.4, 'p_col2_u1':0.3, 'p_col3_u1':0.2, 'p_col4_u1':0.1, 'p_col1_u2':0.1, 'p_col2_u2':0.2, 'p_col3_u2':0.3, 'p_col4_u2':0.4},
             (3,2):{'p_col1_u1':0.7, 'p_col2_u1':0.3, 'p_col1_u2':0.5, 'p_col2_u2':0.5, 'p_col1_u3':0.3, 'p_col2_u3':0.7},
             (3,3):{'p_col1_u1':0.7, 'p_col2_u1':0.2, 'p_col3_u1':0.1, 'p_col1_u2':0.1, 'p_col2_u2':0.7, 'p_col3_u2':0.2, 'p_col1_u3':0.2, 'p_col2_u3':0.1, 'p_col3_u3':0.7},
             (3,4):{'p_col1_u1':0.4, 'p_col2_u1':0.3, 'p_col3_u1':0.2, 'p_col4_u1':0.1, 'p_col1_u2':0.3, 'p_col2_u2':0.2, 'p_col3_u2':0.2, 'p_col4_u2':0.3, 'p_col1_u3':0.1, 'p_col2_u3':0.2, 'p_col3_u3':0.3, 'p_col4_u3':0.4},
             (4,2):{'p_col1_u1':0.7, 'p_col2_u1':0.3, 'p_col1_u2':0.6, 'p_col2_u2':0.4, 'p_col1_u3':0.4, 'p_col2_u3':0.6, 'p_col1_u4':0.3, 'p_col2_u4':0.7},
             (4,3):{'p_col1_u1':0.7, 'p_col2_u1':0.2, 'p_col3_u1':0.1, 'p_col1_u2':0.1, 'p_col2_u2':0.7, 'p_col3_u2':0.2, 'p_col1_u3':0.2, 'p_col2_u3':0.1, 'p_col3_u3':0.7, 'p_col1_u4':0.3, 'p_col2_u4':0.4, 'p_col3_u4':0.3},
             (4,4):{'p_col1_u1':0.4, 'p_col2_u1':0.3, 'p_col3_u1':0.2, 'p_col4_u1':0.1, 'p_col1_u2':0.3, 'p_col2_u2':0.4, 'p_col3_u2':0.1, 'p_col4_u2':0.2, 'p_col1_u3':0.2, 'p_col2_u3':0.1, 'p_col3_u3':0.4, 'p_col4_u3':0.3, 'p_col1_u4':0.1, 'p_col2_u4':0.2, 'p_col3_u4':0.3, 'p_col4_u4':0.4}
}

# Floating parameters
max_urns = 4
max_colours = 4



In [3]:
def create_urnTable_json(urns, colours, balls, ball_comp, name_cols, name_urns):
    """
    Create a JSON file with the urn composition for a given number of urns and colours
    Uninformative prior is used

    urns: number of urns
    colours: number of colours
    balls: number of balls
    ball_comp: dictionary with the ball composition
    name_cols: list with the names of the colours
    name_urns: list with the names of the urns
    """

    # covert decimal prior to fraction
    prior = str(Fraction(1, urns))

    # Create a dataframe from the ball composition dictionary
    ball_comp_df = pd.DataFrame(ball_comp[(urns,colours)], index=[0])
    ball_comp_freq = ball_comp_df * balls

    # urn_lis is a list of urn compositions
    urn_lis = []
    for i in range(urns):
        lis =[]
        for j in range(colours):
            lis.append(str(int(ball_comp_freq['p_col'+str(j+1)+'_u'+str(i+1)][0])) + name_cols[j])
        urn_lis.append(lis)

    # Create a list of dictionaries, each dictionary represents an urn
    urn_entries = []
    for i in range(urns):
        urn_dict = {}
        urn_dict['urnName'] = name_urns[i]
        urn_dict['prior'] = prior
        urn_dict['composition'] = urn_lis[i]
        urn_dict['balls'] = balls
        urn_entries.append(urn_dict)

    # Create a dictionary for the total urn
    total_dict = {}
    total_dict['urnName'] = 'Total'
    total_dict['prior'] = '1'
    # calculate the total composition
    total_comp_lis = []
    for j in range(colours):
        total_balls_per_colour = 0
        name = name_cols[j]
        for i in range(urns):
            total_balls_per_colour += int(ball_comp_freq['p_col'+str(j+1)+'_u'+str(i+1)][0])
        total_comp_lis.append(str(total_balls_per_colour) + name)
    total_dict['composition'] = total_comp_lis
    total_dict['balls'] = balls * urns
    urn_entries.append(total_dict)


    # Wrap the list in an object
    urn_entries_object = {
        "urnEntries": urn_entries
    }

    # Convert the dictionary to a JSON string
    urn_entries_json = json.dumps(urn_entries_object, indent=4)

    # Write the JSON string to a file
    filename = f'urn_entries_{urns}_{colours}.json'
    with open(filename, 'w') as json_file:
        json_file.write(urn_entries_json)


In [4]:
for urns in range(2, max_urns+1):
    for colours in range(2, max_colours+1):
        create_urnTable_json(urns, colours, balls, ball_comp, name_cols, name_urns) 
